In [5]:
# the imports
# the imports
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [6]:
client = MongoClient('localhost:27017')
db = client.venmo
venmo = db.venmo

In [7]:
venmo.count_documents({}) 

7076585

In [8]:
# one random sample from the dataset
def print_schema(obj, level=0):
    if type(obj) == type({}):
        for key, value in obj.items():
            if type(value) == type({}):
                break
        else:
            print('\t' * (level + 1)  + '{', end=' ')
            keys = list(obj.keys())
            for key in keys[:-1]:
                print(key, end=', ')
            print(keys[-1] + '}')
            return
        
        print('\t' * level + '{', end='\n')
        for key, value in obj.items():
            print('\t' * (level + 1) + key + ': ')
            if type(value) == type({}):
                print_schema(value, level+1)
        print('\t' * level + '}')

transactions = venmo.aggregate([{ '$sample' : {'size' : 1}}])
for transaction in transactions:
    print_schema(transaction)

{
	_id: 
	date_updated: 
	transfer: 
	app: 
		{ id, image_url, description, name, site_url}
	comments: 
		{ count, data}
	id: 
	note: 
	audience: 
	likes: 
		{ count, data}
	date_created: 
	mentions: 
		{ count, data}
	type: 
	payment: 
	{
		status: 
		merchant_split_purchase: 
		date_completed: 
		target: 
		{
			merchant: 
			redeemable_target: 
			phone: 
			user: 
				{ username, about, last_name, display_name, friends_count, is_group, is_active, trust_request, email, phone, profile_picture_url, first_name, friend_status, is_blocked, id, identity, date_joined}
			type: 
			email: 
		}
		amount: 
		actor: 
			{ username, about, last_name, display_name, friends_count, is_group, is_active, trust_request, email, phone, profile_picture_url, first_name, friend_status, is_blocked, id, identity, date_joined}
		note: 
		audience: 
		action: 
		date_created: 
		date_reminded: 
		id: 
		date_authorized: 
	}
	authorization: 
}


In [9]:
## aggregate by different types of apps used
transactions =  venmo.aggregate( [{ '$group' : { '_id' : {'description': '$app.description', 'url' : '$app.site_url'} , 'count' : { '$sum' : 1}}}])
df = pd.DataFrame((item['_id']['description'], item['_id']['url'], item['count']) for item in transactions)
df.columns=['Description', 'URL', 'Count']
df.sort_values(by=['Count'], ascending=False).style.hide_index()

Description,URL,Count
Venmo for iPhone,None,6156264
Venmo for Android,None,884654
"Share bills and IOUs. Make sure everyone gets paid back. Totally free for web, iPhone, and Android.",https://secure.splitwise.com/,25621
Venmo.com,None,9271
Payouts API,None,484
Makes splitting restaurant/bar bills easy; facilitates payments between people in the group.,http://www.tabapp.co/,201
Venmo Developer,None,61
Used for Bottle Rocket Utility,None,6
Automation,None,6
Transfer money to phone contacts in a swipe,None,6


In [10]:
## aggregate by email
transactions =  venmo.aggregate( [{ '$group' : { '_id' : {'email': '$payment.actor.email'} , 'count' : { '$sum' : 1}}}])
df = pd.DataFrame((item['_id']['email'], item['count']) for item in transactions)
df.columns=['Email', 'Count']
df.sort_values(by=['Count'], ascending=False).style.hide_index()

Email,Count
None,7076585


In [11]:
## aggregate by payment amount
transactions =  venmo.aggregate( [{ '$group' : { '_id' : {'amount': '$payment.amount'} , 'count' : { '$sum' : 1}}}])
df = pd.DataFrame((item['_id']['amount'], item['count']) for item in transactions)
df.columns=['Amount', 'Count']
df.sort_values(by=['Amount'], ascending=False).style.hide_index()

Amount,Count
None,7076585


In [ ]:
## The count of unique username that paid
transactions =  venmo.aggregate([{'$project' : {'username' : '$payment.actor.username'}}])
df = pd.DataFrame(item['username'] for item in transactions if 'username' in item.keys())
df.columns = ['username']
len(df.username.unique())